In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'C:\Users\MR068144\Downloads\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import shutil
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-08-01 16:18:28


In [2]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2012,2017)
MaxOpenProcesses = 15
Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening'
ReceivedDir  = Project_DIR + '\\2.Received_Data'
Project_GDB  = Project_DIR + '\\5.NM_NS_GDB.gdb'

Src_Loc  = ReceivedDir + '\\crash_asof_11jun2018.csv'
Src_Unit = ReceivedDir + '\\vehicle_asof_11jun2018.csv'
Src_Occ  = ReceivedDir + '\\occupant_asof_11jun2018.csv'

HPMS105 = {year:ReceivedDir + '\\v105\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}
HPMS10  = {year:ReceivedDir + '\\v10\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}

Routes  = {year:Project_GDB + '\\Routes_{}'.format(year) for year in Years}
AttTabs = {year:Project_GDB + '\\SegAtt_{}'.format(year) for year in Years}
Segs    = {year:Project_GDB + '\\Seg_{}'.format(year) for year in Years}
CrashData = {year:Project_GDB + '\\Crash_{}'.format(year) for year in Years}

UTM13N = arcpy.SpatialReference(26913)
NAD83_NM = arcpy.SpatialReference(6529)

RouteID = "ROUTE_ID"
BEG_STA = 'BEG_POINT'
END_STA = 'END_POINT'
Fields = ['F_SYSTEM','THROUGH_LANES','FACILITY_TYPE','AADT','MEDIAN_TYPE','SPEED_LIMIT','LANE_WIDTH']

BridgeUnder = ReceivedDir + '\\v10\\jg_request.gdb\\BridgeUnder'
BridgeLRS   = ReceivedDir + '\\v10\\jg_request.gdb\\BridgeLRS'
Signals     = ReceivedDir + '\\v10\\jg_request.gdb\\SignalizedIntersections'

2018-08-01 16:18:29


In [4]:
Att_Dict = {year:hsmpy3.common.AttributeTabletoDF(AttTabs[year]) for year in Years}
for year in Years:
    
    Att_Dict[year] = Att_Dict[year].sort_values(by=['ROUTE_ID','BEG_POINT'])
    Att_Dict[year]['Len'] = Att_Dict[year].END_POINT-Att_Dict[year].BEG_POINT
    Att_Dict[year] = Att_Dict[year][(Att_Dict[year].Len>0)]
    Att_Dict[year]['SurfWid'] = 24.0
    for i,r in Att_Dict[year].iterrows():
        nl = 2
        try:
            nl = int(r.THROUGH_LANES)
        except:
            pass
        lw = 12.0
        try:
            lw = float(r.LANE_WIDTH)
        except:
            pass
        mdw = {1:0,2:4,3:4,4:6,5:6,6:6,7:6,0:0}
        md = 1
        try:
            md = int(r.MEDIAN_TYPE)
        except:
            pass
        surfwid = max(24,nl*lw+mdw[md])
        Att_Dict[year].set_value(i,'SurfWid',surfwid)

        aadt = r.AADT
        try:
            aadt = int(aadt)
        except:
            aadt = 0
        Att_Dict[year].set_value(i,'AADT',aadt)

In [5]:
Route_Dict = {year:hsmpy3.common.AttributeTabletoDF(Routes[year],True) for year in Years}

In [7]:
Int_DF = hsmpy3.common.AttributeTabletoDF(Project_GDB+'\\CreateInt_Final',True)
Int_DF

,OBJECTID,Shape,Int_ID,Signal,Legs
0,1,(1335392.81956016 783123.279488988 NaN NaN),1,0,3
1,2,(1334144.50315916 778518.588895239 NaN NaN),2,0,4
2,3,(1328611.0706565 773450.040633403 NaN NaN),3,0,3
3,4,(1334159.01001991 779724.156037904 NaN NaN),4,1,4
4,5,(1331152.18270025 775479.166496404 NaN NaN),5,0,3
5,6,(1329105.25897258 773480.37849924 NaN NaN),6,0,3
6,7,(1334115.7919305 776182.70281899 NaN NaN),7,0,3
7,8,(1322078.63457441 771918.929522403 NaN NaN),8,0,3
8,9,(1322413.61356325 772090.92688182 NaN NaN),9,0,3
9,10,(1322801.47564841 772285.332332909 NaN NaN),10,0,3


In [11]:
year = 2012
EventTable = hsmpy3.common.CreateOutPath(MainFile=Project_GDB + '\\Int_'+str(year),appendix='EventTable',Extension='')
arcpy.LocateFeaturesAlongRoutes_lr(
        in_features                = Project_GDB + '\\CreateInt_Final', 
        in_routes                = Routes[year], 
        route_id_field            = 'ROUTE_ID', 
        radius_or_tolerance        = '100 Feet', 
        out_table                = EventTable, 
        out_event_properties    = " ".join(['ROUTE_ID', "POINT", "MP"]),
        route_locations            = "ALL", 
        in_fields                = "FIELDS", 
        m_direction_offsetting    = "M_DIRECTON"
)
df = hsmpy3.common.AttributeTabletoDF(EventTable)
df

,OBJECTID,ROUTE_ID,MP,Distance,Int_ID,Signal,Legs
0,1,BL-11-M,4.311300,-10.453565,1,0,3
1,2,BL-11-P,4.671288,31.129176,1,0,3
2,3,BL-11-M,3.548100,-16.246332,2,0,4
3,4,BL-11-P,3.839300,25.962292,2,0,4
4,5,BL-11-M,1.969000,-26.083795,3,0,3
5,6,BL-11-P,2.112400,22.035417,3,0,3
6,7,FL-4870-P,0.006695,-3.054111,3,0,3
7,8,BL-11-M,3.735100,-20.640934,4,1,4
8,9,BL-11-P,4.043800,20.637028,4,1,4
9,10,FL-4873-P,0.273000,-2.097830,4,1,4


In [ ]:
year = 2012
EventTable = hsmpy3.common.CreateOutPath(MainFile=Project_GDB + '\\Int_'+str(year),appendix='SegEventTable',Extension='')
arcpy.LocateFeaturesAlongRoutes_lr(
        in_features                = Project_GDB + '\\CreateInt_Final', 
        in_routes                = Segs[year], 
        route_id_field            = 'ROUTE_ID', 
        radius_or_tolerance        = '100 Feet', 
        out_table                = EventTable, 
        out_event_properties    = " ".join(['ROUTE_ID', "POINT", "MP"]),
        route_locations            = "ALL", 
        in_fields                = "FIELDS", 
        m_direction_offsetting    = "M_DIRECTON"
)
df2 = hsmpy3.common.AttributeTabletoDF(EventTable)
df2

In [15]:
Att_Dict[2012]

,OBJECTID,ROUTE_ID,BEG_POINT,END_POINT,F_SYSTEM,FACILITY_TYPE,OWNERSHIP,URBAN_CODE,AADT,THROUGH_LANES,...,LANE_WIDTH,Att1,SPEED_LIMIT,LOC_ERROR,ORIG_FID,Shape_Length,FType,SegID,Len,SurfWid
0,1,BIA-AI0005-P,0.0000,12.5000,6,2,0,0,304,0,...,0.0,0.0,0,NO ERROR,1,63347.969163,,BIA-AI0005-P_0.0000_12.5000,12.5000,24.0
1,2,BIA-AI009A-P,0.0000,39.6250,6,2,0,0,95,0,...,0.0,0.0,0,NO ERROR,2,209220.201430,,BIA-AI009A-P_0.0000_39.6250,39.6250,24.0
2,3,BIA-AI7022-P,0.0000,18.2600,5,2,0,0,58,2,...,0.0,0.0,0,NO ERROR,3,91122.918599,R2U,BIA-AI7022-P_0.0000_18.2600,18.2600,24.0
3,4,BIA-BI0582-P,0.0000,1.8130,6,2,0,0,1768,0,...,0.0,0.0,0,NO ERROR,4,5.946053,,BIA-BI0582-P_0.0000_1.8130,1.8130,24.0
4,5,BL-11-M,0.4610,4.9560,0,6,1,99998,0,0,...,0.0,0.0,0,NO ERROR,5,26343.507672,,BL-11-M_0.4610_4.9560,4.4950,24.0
5,6,BL-11-P,0.0000,0.1700,3,2,1,99999,2393,2,...,0.0,0.0,0,NO ERROR,6,897.754341,R2U,BL-11-P_0.0000_0.1700,0.1700,24.0
6,7,BL-11-P,0.1700,0.4150,3,2,1,99998,2393,2,...,0.0,0.0,0,NO ERROR,7,1255.730487,U2U,BL-11-P_0.1700_0.4150,0.2450,24.0
7,8,BL-11-P,0.4150,0.4610,3,2,1,99998,4499,2,...,0.0,0.0,0,NO ERROR,8,235.569795,U2U,BL-11-P_0.4150_0.4610,0.0460,24.0
8,9,BL-11-P,0.4610,3.5920,3,2,1,99998,4499,4,...,0.0,0.0,0,NO ERROR,9,16138.715772,U4U,BL-11-P_0.4610_3.5920,3.1310,24.0
9,10,BL-11-P,3.5920,3.5990,3,2,1,99998,9945,4,...,0.0,0.0,0,NO ERROR,10,41.270923,U4U,BL-11-P_3.5920_3.5990,0.0070,24.0


In [23]:
#SurfWid , AADT ,firstpointM, last
df['SurfWid'] = 30
df['AADT'] = 0
df['fpM'] = 0
df['lpM'] = 0
Att_DF = Att_Dict[2012]
f = FloatProgress(min=0, max=df.shape[0])
display(f)
for i,r in df.iterrows():
    att_df = Att_DF[(Att_DF['ROUTE_ID']==r.ROUTE_ID) & (Att_DF.BEG_POINT<=r.MP) & (Att_DF.END_POINT>=r.MP)]
    if att_df.shape[0]>0:
        df.set_value(i,'SurfWid',max(list(att_df.SurfWid)))
        df.set_value(i,'AADT',max(list(att_df.AADT)))
    f.value += 1

FloatProgress(value=0.0, max=29416.0)

ValueError: max() arg is an empty sequence

In [ ]:
#SurfWid , AADT ,firstpointM, last
df['SurfWid'] = 0
df['AADT'] = 0
df['fpM'] = 0
df['lpM'] = 0
Att_DF = Att_Dict[2012]
f = FloatProgress(min=0, max=df.shape[0])
display(f)
for i,r in df.iterrows():
    att_df = Att_DF[(Att_DF['ROUTE_ID']==r.ROUTE_ID) & (Att_DF.BEG_POINT<=r.MP) & (Att_DF.END_POINT>=r.MP)]
    df.set_value(i,'SurfWid',max(list(att_df.SurfWid)))
    df.set_value(i,'AADT',max(list(att_df.AADT)))
    f.value += 1

FloatProgress(value=0.0, max=29416.0)

In [17]:
df

,OBJECTID,ROUTE_ID,MP,Distance,Int_ID,Signal,Legs,SurfWid,AADT,fpM,lpM
0,1,BL-11-M,4.311300,-10.453565,1,0,3,0,0,0,0
1,2,BL-11-P,4.671288,31.129176,1,0,3,0,0,0,0
2,3,BL-11-M,3.548100,-16.246332,2,0,4,0,0,0,0
3,4,BL-11-P,3.839300,25.962292,2,0,4,0,0,0,0
4,5,BL-11-M,1.969000,-26.083795,3,0,3,0,0,0,0
5,6,BL-11-P,2.112400,22.035417,3,0,3,0,0,0,0
6,7,FL-4870-P,0.006695,-3.054111,3,0,3,0,0,0,0
7,8,BL-11-M,3.735100,-20.640934,4,1,4,0,0,0,0
8,9,BL-11-P,4.043800,20.637028,4,1,4,0,0,0,0
9,10,FL-4873-P,0.273000,-2.097830,4,1,4,0,0,0,0


In [ ]:
# Create Intx App Event Table
App_DF = df
Routes_DF = Route_Dict[2012]
App_DF['Int_ID2'] = App_DF.Int_ID
App_DF['RID'] = App_DF.ROUTE_ID
print(strftime("%Y-%m-%d %H:%M:%S"))
f = FloatProgress(min=0, max=Routes_DF.shape[0])
display(f)
App_Tab = pd.DataFrame(columns=['RID','BMP','EMP','IntProx','Int_ID2','Major'])
x = 0
for i,r in Routes_DF.iterrows():
    rid = r.ROUTE_ID
    app_all = App_DF[(App_DF.RID==rid) & (App_DF.Int_ID2!=0) & (~pd.isnull(App_DF.MP))]
    mp_List = list(set(app_all.MP))
    mp_List.sort()
    int_id_List = [list(app_all[app_all.MP==mp]['Int_ID2'])[0] for mp in mp_List]
    #if False in [(int_id in Int_DF3.Int_ID2) for int_id in int_id_List]:
    #    continue
    for int_id,mp in zip(int_id_List,mp_List):
        #Int    = Int_DF3[Int_DF3.Int_ID2==int_id]
        app    = App_DF[(App_DF.RID==rid) & (App_DF.Int_ID2==int_id)]
        att    = Att_DF[(Att_DF.ROUTE_ID==rid) & (Att_DF.BEG_POINT<=mp) & (Att_DF.END_POINT>=mp)]
        app_x  = App_DF[(App_DF.Int_ID2==int_id) & (App_DF.RID!=rid)]
        att_x = pd.DataFrame()
        for n,m in app_x.iterrows():
            x_rid = m.RID 
            x_mp  = m.MP
            att_x = pd.concat([att_x,Att_DF[(Att_DF.ROUTE_ID==x_rid) & (Att_DF.BEG_POINT<=x_mp) & (Att_DF.END_POINT>=x_mp)]])
        
        int_w = [24.0]
        int_w.extend(list(att_x.SurfWid))
        int_w = float(max(int_w))/5280.0
                
        int_b = 250.0/5280.0
        Major = False
        
        AADT = [0]
        AADT.extend(list(att.AADT))
        AADT = max(AADT)

        AADT_x = [0]
        AADT_x.extend(list(att_x.AADT))
        AADT_x = max(AADT_x)
        
        if AADT>AADT_x:
            Major = True
            
        if Int.Signal.item() and Major:
            int_b = 400.0/5280.0 + int_w
        if Int.Signal.item() and not Major:
            int_b = 250.0/5280.0 + int_w
        if not Int.Signal.item() and Major:
            int_b = 150.0/5280.0 + int_w
        if not Int.Signal.item() and not Major:
            int_b = 90.0/5280.0 + int_w

        bmp = max(mp - int_w - int_b,Routes_DF.loc[i]['Shape'].firstPoint.M)
        emp = max(mp - int_w,Routes_DF.loc[i]['Shape'].firstPoint.M)
        if emp>bmp:
            x += 1
            App_Tab.loc[x] = [rid,bmp,emp,1,int_id,{True:1,False:0}[Major]]

        bmp = max(mp - int_w,Routes_DF.loc[i]['Shape'].firstPoint.M)
        emp = min(mp + int_w,Routes_DF.loc[i]['Shape'].lastPoint.M)
        if emp>bmp:
            x += 1
            App_Tab.loc[x] = [rid,bmp,emp,2,int_id,{True:1,False:0}[Major]]
        
        bmp = min(mp + int_w,Routes_DF.loc[i]['Shape'].lastPoint.M)
        emp = min(mp + int_w + int_b,Routes_DF.loc[i]['Shape'].lastPoint.M)
        if emp>bmp:
            x += 1
            App_Tab.loc[x] = [rid,bmp,emp,1,int_id,{True:1,False:0}[Major]]
    f.value += 1
App_Tab.to_csv('App_Tab{}.csv'.format(strftime("%Y%m%d_%H%M%S")))
App_Tab

In [44]:
# Convert DF to Int FC
DF2FC = hsmpy3.common.CreateOutPath(Project_GDB + '\\CreateInt','Final','')
arcpy.management.CreateFeatureclass(out_name=os.path.basename(DF2FC),
                                    out_path=Project_GDB,geometry_type='POINT',spatial_reference=NAD83_NM)
arcpy.management.AddField(DF2FC,'Int_ID','LONG')
#arcpy.management.AddField(DF2FC,'AppList','TEXT',field_length=900)
#arcpy.management.AddField(DF2FC,'BridgeList','TEXT')
#arcpy.management.AddField(DF2FC,'AppList3','TEXT')
#arcpy.management.AddField(DF2FC,'AppAngle','TEXT')
arcpy.management.AddField(DF2FC,'Signal','Short')
arcpy.management.AddField(DF2FC,'Legs'     ,'Short')
#arcpy.management.AddField(DF2FC,'Selected','Short')
f = FloatProgress(min=0, max=Int_DF3.shape[0],description='Insert Cursor')
display(f)

ic = arcpy.InsertCursor(DF2FC)
for i,r in Int_DF3.iterrows():
    if 1>0:
        row = ic.newRow()
        row.setValue('Int_ID',r.Int_ID2)
        #try:
        #    row.setValue('AppList',r.App1)
        #except:pass
        #row.setValue('BridgeList',str(r.BridgeList))
        #row.setValue('AppList3',str(r.App3))
        row.setValue('Signal',{True:1,False:0}[r.Signal])
        row.setValue('Legs',r.Legs)
        #row.setValue('AppAngle',';'.join(['{}:{:0.1f}'.format(list(d.keys())[0],d[list(d.keys())[0]]) for d in r.AppAngle]))
        #if r.IsInt:
        #    row.setValue('Selected',1)
        #else:
        #    row.setValue('Selected',0)
        p = arcpy.PointGeometry(arcpy.Point(X=r.X,Y=r.Y),UTM13N).projectAs(NAD83_NM)
        row.shape =  p
        ic.insertRow(row)
    f.value += 1
del ic
del row

FloatProgress(value=0.0, description='Insert Cursor', max=103874.0)

In [ ]:
App_Tab = hsmpy3.common.CreateOutPath(Project_GDB + '\\CreateInt','AppTab','')
arcpy.management.CreateTable(out_name=os.path.basename(App_Tab),out_path=Project_GDB)
arcpy.management.AddField(App_Tab,'RID','Text')
arcpy.management.AddField(App_Tab,'MP','Double')
arcpy.management.AddField(App_Tab,'Direction','Text')
arcpy.management.AddField(App_Tab,'Int_ID2','Long')
arcpy.management.AddField(App_Tab,'RID','Text')



